<a href="https://colab.research.google.com/github/Juako1314/Solar-2/blob/main/Solar_calculator1.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install gradio

In [5]:
import gradio as gr
import math as mt

def geometria_solar(dia, hlhr, hlmin, hlseg, GMT, longrados, lonminutos, lonsegundos, es_oes, latgrados, latminutos, latsegundos, Nor_sur):
    hora_local = hlhr + (hlmin/60) + (hlseg / 3600)
    Longitud = longrados + (lonminutos / 60) + (lonsegundos / 3600)
    Latitud = latgrados + (latminutos / 60) + (latsegundos / 3600)

    #Calcula la fraccion de año en radianes
    año_fraccional =((2*mt.pi)/365)*(dia-1+((hora_local-12)/24))

    #Calcula la declinacion solar
    Declinacion_solar_rad =0.006918-0.399912*mt.cos(año_fraccional)+0.070257*mt.sin(año_fraccional)-0.006758*mt.cos(2*año_fraccional)+0.000907*mt.sin(2*año_fraccional)-0.002697*mt.cos(3*año_fraccional)+0.00148*mt.sin(3*año_fraccional)
    Declinacion_solar_grados = Declinacion_solar_rad * (180/mt.pi)

    #Calcula la ecuacion del tiempo
    Ecuacion_del_tiempo = 229.18*(0.000075+0.001868*mt.cos(año_fraccional)-0.032077*mt.sin(año_fraccional)-0.014615*mt.cos(2*año_fraccional)-0.040849*mt.sin(2*año_fraccional))
    min = int(Ecuacion_del_tiempo)
    segs = int((Ecuacion_del_tiempo - min) * 60)

    if es_oes == True:
        nl = Longitud
    else:
        nl = -Longitud

    offset = Ecuacion_del_tiempo+4*nl-60*GMT

    #Calcula el tiempo solar
    Tiempo_solar = hlhr*60 + hlmin + hlseg/60 + offset
    tshr = int(Tiempo_solar/60)
    tsmin = int((Tiempo_solar/60 - int(Tiempo_solar/60))*60)
    tsseg = int(((Tiempo_solar/60 - int(Tiempo_solar/60))*60 - int((Tiempo_solar/60 - int(Tiempo_solar/60))*60))*60)

    #Hora angular
    Hora_angular_grados = (Tiempo_solar/4)-180
    Hora_angular_rad = Hora_angular_grados * (mt.pi/180)

    if Nor_sur == True:
        nlat = Latitud
    else:
        nlat = -Latitud

    #Calcula el angulo cenital
    Angulo_cenital_rad = mt.acos((mt.sin(nlat*(mt.pi/180))*mt.sin(Declinacion_solar_rad))+mt.cos(nlat*(mt.pi/180))*mt.cos(Declinacion_solar_rad)*mt.cos(Hora_angular_rad))
    Angulo_cenital_grados = Angulo_cenital_rad * (180/mt.pi)

    #Calcula el angulo acimutal
    Angulo_acimutal_grados = -(mt.acos((mt.sin(nlat*(mt.pi/180))*mt.cos(Angulo_cenital_rad) - mt.sin(Declinacion_solar_rad))/(mt.cos(nlat*(mt.pi/180))*mt.sin(Angulo_cenital_rad))))*(180/mt.pi) + 180

    #Calcula el amanecer
    ha = (mt.acos((mt.cos(90.833*(mt.pi/180))/(mt.cos(nlat*(mt.pi/180))*mt.cos(Declinacion_solar_rad)))-mt.tan(nlat*(mt.pi/180))*mt.tan(Declinacion_solar_rad)))*(180/mt.pi)
    Amanecer = (720 - 4 * (nl + ha) - Ecuacion_del_tiempo) + (GMT * 60)
    Amhr = int(Amanecer/60)
    Ammin = int((Amanecer/60 - int(Amanecer/60))*60)
    Amseg = int(((Amanecer/60 - int(Amanecer/60))*60 - int((Amanecer/60 - int(Amanecer/60))*60))*60)

    #Calcula el atardecer
    haa = (-mt.acos((mt.cos(90.833*(mt.pi/180))/(mt.cos(nlat*(mt.pi/180))*mt.cos(Declinacion_solar_rad)))-mt.tan(nlat*(mt.pi/180))*mt.tan(Declinacion_solar_rad)))*(180/mt.pi)
    Atardecer = (720 - 4 * (nl + haa) - Ecuacion_del_tiempo) + (GMT * 60)
    Athr = int(Atardecer/60)
    Atmin = int((Atardecer/60 - int(Atardecer/60))*60)
    Atseg = int(((Atardecer/60 - int(Atardecer/60))*60 - int((Atardecer/60 - int(Atardecer/60))*60))*60)
    #Calcula las 12 solares
    Mediodia = (720 - 4 * nl - Ecuacion_del_tiempo) + (GMT * 60)
    Mehr = int(Mediodia/60)
    Memin = int((Mediodia/60 - int(Mediodia/60))*60)
    Meseg = int(((Mediodia/60 - int(Mediodia/60))*60 - int((Mediodia/60 - int(Mediodia/60))*60))*60)

    #Calcula los angulos de inclinacion tomando en cuenta solamente la radiacion directa
    l = []
    x = 0
    for i in range(1, 361):
        fracc =((2*mt.pi)/365)*(i-1+((hora_local-12)/24))
        dsg = (0.006918-0.399912*mt.cos(fracc)+0.070257*mt.sin(fracc)-0.006758*mt.cos(2*fracc)+0.000907*mt.sin(2*fracc)-0.002697*mt.cos(3*fracc)+0.00148*mt.sin(3*fracc))*(180/mt.pi)
        x += dsg

        if i % 30 == 0:
            prom = x / 30
            l.append(prom)
            x = 0
        else:
            continue
    print(l)
    for i in range(12):
        l[i] = nlat - l[i]
    print(l)

    Enero = l[0]
    Febrero = l[1]
    Marzo = l[2]
    Abril = l[3]
    Mayo = l[4]
    Junio = l[5]
    Julio = l[6]
    Agosto = l[7]
    Septiembre = l[8]
    Octubre = l[9]
    Noviembre = l[10]
    Diciembre = l[11]



    return round(año_fraccional,4), round(Declinacion_solar_grados,4), min, segs, tshr, tsmin, tsseg, round(Hora_angular_grados, 4), round(Angulo_cenital_grados, 4), Angulo_acimutal_grados, Amhr, Ammin, Amseg, Athr, Atmin, Atseg, Mehr, Memin, Meseg, Enero, Febrero, Marzo, Abril, Mayo, Junio, Julio, Agosto, Septiembre, Octubre, Noviembre, Diciembre

demo = gr.Interface(fn=geometria_solar,
                   inputs=[gr.Number(label="Dia", info = "Ingrese un dia entre 1 y 365"),
                           gr.Number(label = "Hora local (Hr)", info = ""), gr.Number(label = "Hora local (min)"),
                           gr.Number(label = "Hora local (seg)"), gr.Number(label = "GMT", info = "Ej. GMT-7, solo ingrese el '-7'"),
                           gr.Number(label = "Longitud (°)"), gr.Number(label = "Longitud (min)"), gr.Number(label = "Longitud(seg)"),
                           gr.Checkbox(label = "Este", value = True, interactive = True),
                           gr.Number(label = "Latitud (°)"), gr.Number(label = "Latitud (min)"), gr.Number(label = "Latitud(seg)"),
                           gr.Checkbox(label = "Norte", value = True, interactive = True)
                           ],
                   outputs=[
                            gr.Number(label="Año fraccional (Rad)"), gr.Number(label="Declinacion solar (Grados)"),
                            gr.Number(label = "Ecuacion del tiempo (min)"), gr.Number(label = "Ecuacion del tiempo (seg)"),
                            gr.Number(label = "Tiempo solar (Hr)"), gr.Number(label = "Tiempo solar (min)"),
                            gr.Number(label = "Tiempo solar (seg)"), gr.Number(label = "Hora angular (°)"),
                            gr.Number(label = "Angulo cenital (Grados)"),
                            gr.Number(label = "Angulo acimutal (Grados)", info = "Tomando al norte como referencia"),
                            gr.Number(label = "Amanecer (Hr)"), gr.Number(label = "Amanecer (min)"),
                            gr.Number(label = "Amanecer (seg)"), gr.Number(label = "Atardecer (Hr)"), gr.Number(label = "Atardecer (min)"),
                            gr.Number(label = "Atardecer (seg)"), gr.Number(label = "Mediodia solar (Hr)"),
                            gr.Number(label = "Mediodia solar (min)"),
                            gr.Number(label = "Mediodia solar (seg)"),
                            gr.Number(label = "Angulo de inclinacion optimo (Enero)", info = "Solamente se toma en cuenta la radiacion directa"),
                            gr.Number(label = "Angulo de inclinacion optimo (Febrero)", info = "Solamente se toma en cuenta la radiacion directa"),
                            gr.Number(label = "Angulo de inclinacion optimo (Marzo)", info = "Solamente se toma en cuenta la radiacion directa"),
                            gr.Number(label = "Angulo de inclinacion optimo (Abril)", info = "Solamente se toma en cuenta la radiacion directa"),
                            gr.Number(label = "Angulo de inclinacion optimo (Mayo)", info = "Solamente se toma en cuenta la radiacion directa"),
                            gr.Number(label = "Angulo de inclinacion optimo (Junio)", info = "Solamente se toma en cuenta la radiacion directa"),
                            gr.Number(label = "Angulo de inclinacion optimo (Julio)", info = "Solamente se toma en cuenta la radiacion directa"),
                            gr.Number(label = "Angulo de inclinacion optimo (Agosto)", info = "Solamente se toma en cuenta la radiacion directa"),
                            gr.Number(label = "Angulo de inclinacion optimo (Septiembre)", info = "Solamente se toma en cuenta la radiacion directa"),
                            gr.Number(label = "Angulo de inclinacion optimo (Octubre)", info = "Solamente se toma en cuenta la radiacion directa"),
                            gr.Number(label = "Angulo de inclinacion optimo (Noviembre)", info = "Solamente se toma en cuenta la radiacion directa"),
                            gr.Number(label = "Angulo de inclinacion optimo (Diciembre)", info = "Solamente se toma en cuenta la radiacion directa")
                           ]
                    )
demo.launch(share = True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://db672cd2d4852c65b0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
